# Handling 'Medium' Data with Dask - Explore SA Gawler Challenge

### In my previous notebooks we've looked at data cleaning, feature engineering and machine learning on geological datasets. All of these example datasets can be considered small in terms of 'big data', but for more informative data analysis (or ML models) it's necessary to increase the amount of data we use. However, we reach a problem when incorporating more and more data into our workflows; computing power. This notebook looks at how to deal with this issue, using the python package Dask to perform large data processes with a little laptop and providing a couple of helpful python hints along the way.

<img src="Images/Intro.png" />

Firstly, let's define what big, medium and small data is in terms of size. A lot of what I've read online characterizes data as 'Big Data' when it contains high volume of data, a high variety in data types and slow velocity when it comes to processing (known as the 3 V's). A general rule of thumb I like is that small data is <10GB, medium data is <1TB and anything over 1TB is considered big data. As mentioned above, all previous notebook datasets have been within the 'small data' category and could be created/processed using python packages such as pandas, numpy and sklearn, but how do we do this for data that is too large for my RAM? 

The objective of this notebook is to use python to manufacture a 'medium data' sized DataFrame that is ready undergo data analysis and create Machine Learning models on my laptop.

## The Data

With the [Unearthed ExploreSA Gawler](https://unearthed.solutions/u/exploresa-gawler) challenge coming up, I thought it'd a great opportunity to utilize spatial data from [SARIG](https://map.sarig.sa.gov.au/) for the example dataset. For this, I selected a large northern chunk of the Gawler Craton to focus on which incorporated 36 different map zones with a total area of 582,918.7 km^2. From SARIG, I downloaded a whole bunch of different geological data spanning from the Archaen basement lithology to surface regolith material. Using QGIS, I chose 22 features (listed below) that I wanted to put into a workable DataFrame. Keep in mind the selection of these features were chosen purely for this example and may or may not win you the Gawler Challenge.

<img src="Images/Table.png" />

The way I chose to create this dataset was by converting QGIS vector files into images and then loading the image pixels as rows and columns to a DataFrame. First I clipped the data to my selected study area and then converted the files to raster, with each pixel representing a 50m x 50m area. Because the study area covers 582,918.7 km^2, each feature is composed of 242,832,800 pixels, we're definitely venturing into the realm of big data now. I saved each image as a ~1GB .tif file, ready to be loaded into python.

The GIF below shows the study area and a snippet of the features used within this notebook.

![ ](Images/gawlerexample.gif)

## Loading in the Data - Dask Array

With smaller datasets pandas can be used to create DataFrames, do feature engineering etc. but when we are dealing with 22 separate image files at 1GB each pandas just doesn't quite cut it. This is because pandas tries to combine all the files into one DataFrame and store it in my computers memory (or RAM). My computer has a RAM of 8GB and the file I'm trying to create is 22GB. This is where python package Dask comes into play. The way Dask operates is better explained [here](https://www.analyticsvidhya.com/blog/2018/08/dask-big-datasets-machine_learning-python/), but it effectively splits large data files into multiple partitions and works on one partition at a time. Another great thing about Dask is that it is built on common python packages pandas, numpy and scikit learn, which means if you're already familiar with these packages it will be easy to understand Dask. Let's get started in setting up our DataFrame.

In [1]:
#Importing all required packages
import numpy as np
import pandas as pd

import skimage

import glob
import os

import dask
import dask.array as da
import dask.dataframe as dd

After importing the python libraries our next step is to load up our image files. To do this we will use dask.delayed to 'lazily' load in the images as Dask arrays. Lazily loading something as an array just means we get the data type of the file and the shape of it but don't actually do anything with the file yet. 

As we want to turn these images into columns we must 'flatten' them. This means lining up all the pixels in a row as a 1 dimensional array. After we have lazily loaded the images, we can convert them into Dask arrays and then stack them all together into the shape of the DataFrame we want (as columns).

In [2]:
#Preparing to use skimage to lazily load in files
imread = dask.delayed(skimage.io.imread, pure=True)

#Reading the files and flattening them
filenames = sorted(glob.glob(" >>FILEPATH FOR TIFF IMAGES<<< "))
lazy_images = [imread(path) for path in filenames]
lazy_flat = [i.flatten() for i in lazy_images]

#Gaining the shape of the arrays
sample = lazy_flat[0].compute()

#Creating dask arrays from all the read images
arrays = [da.from_delayed(lazy_image, dtype=sample.dtype, shape=sample.shape) for lazy_image in lazy_flat]

#Stacking all the arrays together
stack = da.stack(arrays, axis=1)

After the above cell is executed, we will have a stack of all our 1D image arrays. Before we go anywhere though, lets load in the image names. NOTE: it is important to make sure the right names match up with the right arrays, best bet is to sort everything alphabetically before loading in.

In [3]:
#Gaining the names of the files and removing the file extension
names = sorted(os.listdir(" >>FILEPATH FOR TIFF IMAGES<<< "))
names = [s.replace('.tif', '') for s in names]

Now let's have a look at the shape of our stacked array.

In [4]:
stack

dask.array<stack, shape=(242832800, 22), dtype=float32, chunksize=(242832800, 1), chunktype=numpy.ndarray>

In [5]:
stack.shape

(242832800, 22)

Here we can see that the shape of our stacked arrays are 22 arrays of 242,832,800 values. Our next step is to convert this into a Dask DataFrame so we can do some work on it. To do this we can just convert it straight from the stack of arrays, using the file names as column headers.

## Creating the DataFrame- Dask DataFrame

In [ ]:
#Creating a dask DataFrame from the array
df = dd.from_dask_array(stack, columns=names)

In [7]:
#Analysing the DataFrame data types
df.describe()

,Arch 200m Fault Buffer,Arch 200m Geology Buffer,Arch Geology,Areas of Interest Points,Calcareous Induration,Dep-Ero-Res,Distance to Craton Scaled,Ferruginous Induration,Gypsiferous Induration,Map Tiles,Mid Meso 200m Fault Buffer,Mid Meso 200m Geology Buffer,Mid Meso Geology,Mixed Calcareous-Gypsiferous Induration,Neo-Ord 200m Fault Buffer,Neo-Ord 200m Geology Buffer,Neo-Ord Geology,Regolith Landform,Regolith Material,Siliceous Induration,Surface Geology 2M,Trans-Insitu
npartitions=1,,,,,,,,,,,,,,,,,,,,,,
0,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
242832799,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


We've now got a fully functioning Dask DataFrame. Notice that npartition = 1, this means that if we were going to compute any processes it would just do it on the whole DataFrame instead of splitting it up into smaller DataFrames. We should change this by repartitioning the DataFrame.

<img src="Images/partition.png" />

In [9]:
#Repartitioning the df into 1000 parts
df = df.repartition(npartitions=1000)

In [10]:
df

,Arch 200m Fault Buffer,Arch 200m Geology Buffer,Arch Geology,Areas of Interest Points,Calcareous Induration,Dep-Ero-Res,Distance to Craton Scaled,Ferruginous Induration,Gypsiferous Induration,Map Tiles,Mid Meso 200m Fault Buffer,Mid Meso 200m Geology Buffer,Mid Meso Geology,Mixed Calcareous-Gypsiferous Induration,Neo-Ord 200m Fault Buffer,Neo-Ord 200m Geology Buffer,Neo-Ord Geology,Regolith Landform,Regolith Material,Siliceous Induration,Surface Geology 2M,Trans-Insitu
npartitions=1000,,,,,,,,,,,,,,,,,,,,,,
0,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
242832,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242589966,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242832799,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


We can see that the Dask DataFrame has now been split up into 1000 smaller DataFrames with approximately 242,832 rows each. This makes it a lot easier for the computer to process the DataFrame as a whole. 

This next step may seem a little strange, but I have noticed that saving these newly partitioned DataFrames to .csv files and reloading them back into the notebook as a new Dask DataFrame significantly decreases processing time. So in a similar fashion to pandas, I will save the 1000 partitioned .csv files to a hard drive and read them back into the workspace.

In [ ]:
#Saving all the partitions to csv to not be running from memory
df.to_csv('  >>FILEPATH FOR CSV FILES<<<  ')

In [12]:
#Reading all csv files into a dask df
dfn = dd.read_csv('  >>FILEPATH FOR CSV FILES<<<  ')

<img src="Images/csv_files.png" />

With the DataFrame loaded back into the notebook let's take a look at the head of it. This only computes on the first partition, unless stated otherwise.

In [13]:
#Looking at the head of the first partition
dfn.head()

,Unnamed: 0,Arch 200m Fault Buffer,Arch 200m Geology Buffer,Arch Geology,Areas of Interest Points,Calcareous Induration,Dep-Ero-Res,Distance to Craton Scaled,Ferruginous Induration,Gypsiferous Induration,...,Mid Meso Geology,Mixed Calcareous-Gypsiferous Induration,Neo-Ord 200m Fault Buffer,Neo-Ord 200m Geology Buffer,Neo-Ord Geology,Regolith Landform,Regolith Material,Siliceous Induration,Surface Geology 2M,Trans-Insitu
0,0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#Looking at the datatypes of each column
dfn.describe()

,Unnamed: 0,Arch 200m Fault Buffer,Arch 200m Geology Buffer,Arch Geology,Areas of Interest Points,Calcareous Induration,Dep-Ero-Res,Distance to Craton Scaled,Ferruginous Induration,Gypsiferous Induration,Map Tiles,Mid Meso 200m Fault Buffer,Mid Meso 200m Geology Buffer,Mid Meso Geology,Mixed Calcareous-Gypsiferous Induration,Neo-Ord 200m Fault Buffer,Neo-Ord 200m Geology Buffer,Neo-Ord Geology,Regolith Landform,Regolith Material,Siliceous Induration,Surface Geology 2M,Trans-Insitu
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


Our DataFrame is now available to be used for any kind of data analysis and feature engineering we see fit to do. As an example lets try to dummy encode our categorical features. 

## Dummy Encode Variables - Dask ML

This is the first time we've looked at encoding in my notebooks, and all it effectively does is convert categorical features to numerical features. The main reason for this is because some machine learning models only take numerical features as inputs. Encoding is quite simple and below image is a visual representation of how dummy encoding works.

<img src="Images/dummies.png" />

We will do the exact same thing to our categorical features in this example. Firstly, let's get a list of all the categorical features we want to get dummies for and convert them to dtype 'object'.

In [15]:
#Creating list of categorical features
cats = ['Arch Geology', 'Calcareous Induration', 'Dep-Ero-Res', 'Ferruginous Induration', 'Gypsiferous Induration', 
        'Mid Meso Geology', 'Mixed Calcareous-Gypsiferous Induration', 'Neo-Ord Geology', 'Regolith Landform', 
        'Regolith Material', 'Siliceous Induration', 'Surface Geology 2M', 'Trans-Insitu']

In [16]:
#Converting the dtypes
for i in cats:
    dfn[i] = dfn[i].astype('str')

In [17]:
dfn.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 23 entries, Unnamed: 0 to Trans-Insitu
dtypes: object(13), float64(9), int64(1)

Here we can our 13 selected features as data type object now. Next we will import a categorizer and dummy encoder then apply them to our data using sklearns pipeline. This is also the first time we've looked at pipelines in these notebooks. Pipeline is a tool that creates a repeatable set of processing steps that can be applied to our data. In this case we will create a two step pipeline, the first step categorizes the object type features (requirement for DummyEncoder) and the second step applies the encoder. We use pipeline instead of doing both of these steps individually because it works simultaneously on the data.

In [19]:
from sklearn.pipeline import make_pipeline
from dask_ml.preprocessing import DummyEncoder, Categorizer

In [20]:
#Making the pipeline
pipe = make_pipeline(Categorizer(), DummyEncoder())

In [21]:
#Fitting the pipeline to the dataframe
pipe.fit(dfn)

Pipeline(memory=None,
         steps=[('categorizer', Categorizer(categories=None, columns=None)),
                ('dummyencoder', DummyEncoder(columns=None, drop_first=False))],
         verbose=False)

In [22]:
#Transforming the dataframe
dft = pipe.transform(dfn)

In [23]:
dft

,Unnamed: 0,Arch 200m Fault Buffer,Arch 200m Geology Buffer,Areas of Interest Points,Distance to Craton Scaled,Map Tiles,Mid Meso 200m Fault Buffer,Mid Meso 200m Geology Buffer,Neo-Ord 200m Fault Buffer,Neo-Ord 200m Geology Buffer,Arch Geology_0.0,Arch Geology_34.0,Arch Geology_32.0,Arch Geology_33.0,Arch Geology_31.0,Arch Geology_35.0,Arch Geology_36.0,Arch Geology_37.0,Arch Geology_38.0,Arch Geology_39.0,Arch Geology_45.0,Arch Geology_11.0,Arch Geology_12.0,Arch Geology_10.0,Arch Geology_49.0,Arch Geology_5.0,Arch Geology_13.0,Arch Geology_9.0,Arch Geology_1.0,Arch Geology_14.0,Arch Geology_48.0,Arch Geology_22.0,Arch Geology_3.0,Arch Geology_7.0,Arch Geology_4.0,Arch Geology_15.0,Arch Geology_42.0,Arch Geology_6.0,Arch Geology_19.0,Arch Geology_46.0,Arch Geology_47.0,Arch Geology_25.0,Arch Geology_30.0,Arch Geology_2.0,Arch Geology_20.0,Arch Geology_16.0,Arch Geology_43.0,Arch Geology_41.0,Arch Geology_26.0,Arch Geology_44.0,Arch Geology_40.0,Arch Geology_24.0,Arch Geology_8.0,Arch Geology_23.0,Arch Geology_51.0,Arch Geology_53.0,Arch Geology_54.0,Arch Geology_21.0,Arch Geology_17.0,Arch Geology_28.0,Arch Geology_27.0,Arch Geology_50.0,Arch Geology_18.0,Arch Geology_29.0,Arch Geology_55.0,Arch Geology_56.0,Arch Geology_52.0,Calcareous Induration_0.0,Calcareous Induration_1.0,Dep-Ero-Res_0.0,Dep-Ero-Res_3.0,Dep-Ero-Res_1.0,Dep-Ero-Res_2.0,Dep-Ero-Res_4.0,Ferruginous Induration_0.0,Ferruginous Induration_1.0,Gypsiferous Induration_0.0,Gypsiferous Induration_1.0,Mid Meso Geology_0.0,Mid Meso Geology_1.0,Mixed Calcareous-Gypsiferous Induration_0.0,Mixed Calcareous-Gypsiferous Induration_1.0,Neo-Ord Geology_0.0,Neo-Ord Geology_3.0,Neo-Ord Geology_2.0,Neo-Ord Geology_20.0,Neo-Ord Geology_1.0,Neo-Ord Geology_17.0,Neo-Ord Geology_19.0,Neo-Ord Geology_18.0,Neo-Ord Geology_16.0,Neo-Ord Geology_13.0,Neo-Ord Geology_24.0,Neo-Ord Geology_14.0,Neo-Ord Geology_21.0,Neo-Ord Geology_6.0,Neo-Ord Geology_15.0,Neo-Ord Geology_4.0,Neo-Ord Geology_5.0,Neo-Ord Geology_9.0,Neo-Ord Geology_22.0,Neo-Ord Geology_25.0,Neo-Ord Geology_23.0,Neo-Ord Geology_12.0,Neo-Ord Geology_11.0,Neo-Ord Geology_26.0,Neo-Ord Geology_10.0,Neo-Ord Geology_7.0,Neo-Ord Geology_8.0,Regolith Landform_0.0,Regolith Landform_7.0,Regolith Landform_5.0,Regolith Landform_1.0,Regolith Landform_2.0,Regolith Landform_6.0,Regolith Landform_8.0,Regolith Landform_3.0,Regolith Landform_4.0,Regolith Landform_9.0,Regolith Material_0.0,Regolith Material_9.0,Regolith Material_5.0,Regolith Material_2.0,Regolith Material_7.0,Regolith Material_4.0,Regolith Material_1.0,Regolith Material_11.0,Regolith Material_3.0,Regolith Material_10.0,Regolith Material_12.0,Regolith Material_8.0,Regolith Material_6.0,Regolith Material_13.0,Siliceous Induration_0.0,Siliceous Induration_1.0,Surface Geology 2M_0.0,Surface Geology 2M_12.0,Surface Geology 2M_1.0,Surface Geology 2M_16.0,Surface Geology 2M_13.0,Surface Geology 2M_9.0,Surface Geology 2M_15.0,Surface Geology 2M_4.0,Surface Geology 2M_5.0,Surface Geology 2M_10.0,Surface Geology 2M_14.0,Surface Geology 2M_6.0,Surface Geology 2M_2.0,Surface Geology 2M_7.0,Surface Geology 2M_19.0,Surface Geology 2M_11.0,Surface Geology 2M_3.0,Surface Geology 2M_8.0,Surface Geology 2M_18.0,Surface Geology 2M_31.0,Surface Geology 2M_17.0,Surface Geology 2M_25.0,Surface Geology 2M_38.0,Surface Geology 2M_26.0,Surface Geology 2M_22.0,Surface Geology 2M_40.0,Surface Geology 2M_30.0,Surface Geology 2M_24.0,Surface Geology 2M_41.0,Surface Geology 2M_21.0,Surface Geology 2M_29.0,Surface Geology 2M_35.0,Surface Geology 2M_34.0,Surface Geology 2M_44.0,Surface Geology 2M_27.0,Surface Geology 2M_33.0,Surface Geology 2M_37.0,Surface Geology 2M_42.0,Surface Geology 2M_36.0,Surface Geology 2M_62.0,Surface Geology 2M_28.0,Surface Geology 2M_32.0,Surface Geology 2M_20.0,Surface Geology 2M_23.0,Surface Geology 2M_39.0,Surface Geology 2M_45.0,Surface Geology 2M_53.0,Surface Geology 2M_43.0,Surface Geology 2M_50.0,Surface Geology 2M_49.0,Surface Geology 2M_48.0,Surface Geology 2M_46.0,Surfac

We now have a DataFrame with all our new dummy encoded features. We have successfully added 240 columns to our DataFrame, making it even bigger than before, without having to buy a brand new super computer. From here we can go onto further feature engineering, data cleaning, other preprocessing and eventually machine learning if we wanted to. 

With Dask we've just collated and run some processes on a large amount of data using nothing but a laptop and a hard drive. I hope this notebook can shed some light on and evoke some ideas of how Dask can be used for handling large datasets. Cheers!